In [1]:
import tensorflow as tf
import numpy as np
# from tf.keras.utils import plot_model 

In [2]:
i = 1
if i:
    %cd ../
    i=0
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from config import Config
from data_utils import video_to_frames
from data_utils import metadata_loader
from data_utils import dataset_builder

from models.c3d import C3D_model, small_c3d, bigger_c3d

plt.style.use('ggplot')
%load_ext autoreload
%autoreload 2
import time

/home/maxdox/Desktop/DeepLearning/Project/video-based-human-activity-recognition


# Model

# 1) Something Something Dataset

In [4]:
# hyperparams
config = Config()

# Decode videos
#video_to_frames.decode_videos(config)

# Get metadata
ml = metadata_loader.MetadataLoader(config)
metadata = ml.load_metadata()

db2 = dataset_builder.DatasetBuilder(config)
train_dataset2 = db2.make_video_dataset(metadata['train'])

def format_example(image, label):
    #image = tf.cast(image, tf.float32)
    image.set_shape((None, config.img_width, config.img_height, 3))
    # image = (image/127.5) - 1
    # image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    label.set_shape([config.n_classes])
    return image, label

train_SMTH = train_dataset2.map(format_example)
train_SMTH

<MapDataset shapes: ((None, 200, 120, 3), (174,)), types: (tf.float32, tf.int32)>

In [5]:
start = time.time()
for x, y in train_SMTH.batch(1).take(10):
    print(x.shape[1:],np.argmax(y.numpy()))
print("\ntime required for video dataset:", time.time()-start)
# x,y = None, None

(85, 200, 120, 3) 55
(85, 200, 120, 3) 105
(85, 200, 120, 3) 141
(85, 200, 120, 3) 11
(85, 200, 120, 3) 14
(85, 200, 120, 3) 44
(85, 200, 120, 3) 30
(85, 200, 120, 3) 157
(85, 200, 120, 3) 59
(85, 200, 120, 3) 159

time required for video dataset: 1.1489386558532715


In [6]:
input_shape = (config.max_frames, config.img_width, config.img_height, 3)
model= small_c3d(input_shape, config.n_classes)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1 (Conv3D)                   (None, 85, 200, 120, 32)  2624      
_________________________________________________________________
2 (MaxPooling3D)             (None, 84, 100, 60, 32)   0         
_________________________________________________________________
3 (Dropout)                  (None, 84, 100, 60, 32)   0         
_________________________________________________________________
4 (Conv3D)                   (None, 84, 100, 60, 64)   55360     
_________________________________________________________________
5 (MaxPooling3D)             (None, 28, 34, 20, 64)    0         
_________________________________________________________________
6 (Dropout)                  (None, 28, 34, 20, 64)    0         
_________________________________________________________________
7 (Conv3D)                   (None, 28, 34, 20, 64)    1

In [7]:
model.fit(train_SMTH.batch(2))

  46/3959 [..............................] - ETA: 34:45:29 - loss: 5.2197 - accuracy: 0.0109

KeyboardInterrupt: 

# 2) KTH  dataset

In [3]:
from data_utils.kth_dataset_builder import DatasetBuilder

# Setup builder
video_path = '../data/kth-actions/video'
frame_path = '../data/kth-actions/frame'
builder = DatasetBuilder(video_path, frame_path, img_width=84, img_height=84, ms_per_frame=100,max_frames=50)

# Convert videos and generate metadata
#builder.convert_videos_to_frames()
metadata = builder.generate_metadata()

# Build datasets
train_dataset = builder.make_video_dataset(metadata=metadata['train'])
valid_dataset = builder.make_video_dataset(metadata=metadata['valid'])


IMG_SIZE = 160 # All images will be resized to 160x160

def format_example(image, label):
    #image = tf.cast(image, tf.float32)
    image.set_shape((None, 84, 84, 1))
    # image = (image/127.5) - 1
    # image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    label.set_shape([6])
    return image, label

train = train_dataset.map(format_example)
validation = valid_dataset.map(format_example)
# test = test_dataset.map(format_example)


# Verify that the datasets work
for vid, label in train.batch(2).take(3):
    print(vid.shape, label.shape)

train

(2, 50, 84, 84, 1) (2, 6)
(2, 50, 84, 84, 1) (2, 6)
(2, 50, 84, 84, 1) (2, 6)


<MapDataset shapes: ((None, 84, 84, 1), (6,)), types: (tf.float32, tf.int32)>

In [8]:
input_shape = (50, 84, 84, 1)
model= bigger_c3d(input_shape, 6)
model.fit(train.batch(2), epochs=1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50, 84, 84, 1)]   0         
_________________________________________________________________
conv3d (Conv3D)              (None, 48, 82, 82, 64)    1792      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 48, 41, 41, 64)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 46, 39, 39, 64)    110656    
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 23, 19, 19, 64)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 21, 17, 17, 64)    110656    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 10, 8, 8, 64)      0     

KeyboardInterrupt: 